<a href="https://colab.research.google.com/github/kipronod/covid19-deep-learning-prediction/blob/master/COVID_19_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import random
import torch
import torch.utils.data
import pandas as pd
import matplotlib.pyplot as plt
import math

# from google.colab import drive
# drive.mount('/content/drive')

# df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data.csv')

In [ ]:
header=df.columns
df.head()
df

,batch_date,test_name,swab_type,covid19_test_results,age,high_risk_exposure_occupation,high_risk_interactions,diabetes,chd,htn,cancer,asthma,copd,autoimmune_dis,smoker,temperature,pulse,sys,dia,rr,sats,rapid_flu_results,rapid_strep_results,ctab,labored_respiration,rhonchi,wheezes,days_since_symptom_onset,cough,cough_severity,fever,sob,sob_severity,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,cxr_findings,cxr_impression,cxr_label,cxr_link,er_referral
0,2020-06-02,"SARS-CoV-2, NAA",Nasal,Negative,37,False,NaN,False,False,False,False,False,False,False,False,36.50,81.0,116.0,77.0,15.0,97.0,NaN,NaN,True,False,NaN,NaN,NaN,False,NaN,False,False,NaN,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,False
1,2020-04-14,Rapid COVID-19 Test,Nasopharyngeal,Negative,52,False,False,False,False,False,False,False,False,False,False,37.10,65.0,132.0,74.0,16.0,100.0,NaN,NaN,False,False,False,False,NaN,False,NaN,NaN,False,NaN,False,False,True,False,False,False,False,False,NaN,NaN,NaN,NaN,False
2,2020-06-16,"SARS-CoV-2, NAA",Nasal,Negative,15,False,False,False,False,False,False,False,False,False,False,37.00,74.0,123.0,80.0,17.0,99.0,NaN,NaN,False,False,False,False,NaN,False,NaN,False,False,NaN,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,False
3,2020-04-28,Rapid COVID-19 Test,Nasopharyngeal,Negative,42,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,False
4,2020-06-02,"SARS-CoV-2, NAA",Nasal,Negative,24,True,NaN,False,False,False,False,False,False,False,False,36.80,61.0,125.0,70.0,15.0,99.0,NaN,NaN,True,False,NaN,NaN,NaN,False,NaN,NaN,False,NaN,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11164,2020-04-28,Rapid COVID-19 Test,Nasopharyngeal,Negative,81,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,False
11165,2020-06-09,SARS COV2 NAAT,Nasal,Negative,48,True,True,False,False,False,False,False,False,False,False,36.70,77.0,145.0,98.0,17.0,99.0,NaN,NaN,True,False,True,True,NaN,False,NaN,False,False,NaN,False,True,False,False,False,False,True,False,NaN,NaN,NaN,NaN,False
11166,2020-05-05,Rapid COVID-19 Test,Nasopharyngeal,Negative,59,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,False
11167,2020-05-12,SARS COV2 NAAT,Nasopharyngeal,Negative,55,False,False,False,False,False,False,False,False,False,False,36.75,90.0,119.0,87.0,15.0,100.0,NaN,NaN,False,False,False,False,NaN,False,NaN,False,False,NaN,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,False


In [ ]:
np.sum(df.isna())

batch_date                           0
test_name                            0
swab_type                            0
covid19_test_results                 0
age                                  0
high_risk_exposure_occupation      169
high_risk_interactions            1501
diabetes                             0
chd                                  0
htn                                  0
cancer                               0
asthma                               0
copd                                 0
autoimmune_dis                       0
smoker                               0
temperature                       4627
pulse                             4644
sys                               4618
dia                               4618
rr                                5407
sats                              4783
rapid_flu_results                11004
rapid_strep_results              11083
ctab                              5293
labored_respiration               4146
rhonchi                  

In [ ]:
# remove irrelevant columns and with many NaN
np.sum(df.isna())
df_ = df.drop(columns=['swab_type', 'batch_date', 'test_name', 'rapid_flu_results','rapid_strep_results','days_since_symptom_onset', 'cough_severity', 
                     'cxr_findings', 'sob_severity', 'cxr_impression', 'cxr_label', 'cxr_link','labored_respiration','wheezes'])

# numerical and categorical columns
num_columns = ["age", "temperature", "pulse", "sys", "dia", "rr", "sats"]
cat_colums = list(set(df_.columns.tolist()) - set(num_columns))

In [ ]:
# one-hot encoding
for i in df_.columns:
  if df_[i].dtypes == bool:
    df_[i]=df_[i].astype(np.float64)
  elif df_[i].dtype == object:
    if i != 'covid19_test_results':
      df_[i]=df_[i].astype(np.float64)

df_['covid19_test_results'] = (df_['covid19_test_results'] == 'Positive').astype(np.float64)
df_

,covid19_test_results,age,high_risk_exposure_occupation,high_risk_interactions,diabetes,chd,htn,cancer,asthma,copd,autoimmune_dis,smoker,temperature,pulse,sys,dia,rr,sats,ctab,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,er_referral
0,0.0,37,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.50,81.0,116.0,77.0,15.0,97.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.10,65.0,132.0,74.0,16.0,100.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.00,74.0,123.0,80.0,17.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,24,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.80,61.0,125.0,70.0,15.0,99.0,1.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11164,0.0,81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11165,0.0,48,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.70,77.0,145.0,98.0,17.0,99.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11166,0.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11167,0.0,55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.75,90.0,119.0,87.0,15.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# normalize numerical data
def data_normalize(x):
  return (x-min(x))/(max(x)-min(x))

for i in num_columns:
  df_[i] = data_normalize(df_[i])

In [ ]:
# divide into positive and negative dataframes
df_positive = df_[df_['covid19_test_results'] == 1.0]
df_negative = df_[df_['covid19_test_results'] == 0.0]

# impute numerical NaN with mean value
for i in num_columns:
  df_positive[i] = df_positive[i].fillna(df_positive[i].mean())
  df_negative[i] = df_negative[i].fillna(df_negative[i].mean())

# impute categorical NaN with most frequent value
for i in cat_colums:
  df_positive = df_positive.apply(lambda x: x.fillna(x.value_counts().index[0]))
  df_negative = df_negative.apply(lambda x: x.fillna(x.value_counts().index[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# merge and shuffle rows
df_combined = pd.concat([df_negative, df_positive])
df_combined = df_combined.sample(frac=1).reset_index(drop=True)

print (len(df_negative), len(df_positive), len(df_combined), len(df))
print ('Negative: ', len(df_negative)/len(df_combined), '%')
print ('Positive: ', len(df_positive)/len(df_combined), '%')
df_combined

10854 315 11169 11169
Negative:  0.9717969379532635 %
Positive:  0.028203062046736505 %


,covid19_test_results,age,high_risk_exposure_occupation,high_risk_interactions,diabetes,chd,htn,cancer,asthma,copd,autoimmune_dis,smoker,temperature,pulse,sys,dia,rr,sats,ctab,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,er_referral
0,1.0,0.516129,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.449438,0.345455,0.306250,0.434343,0.318182,0.916667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.989247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.438202,0.218182,0.406250,0.464646,0.181818,0.921815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.408602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.431170,0.350087,0.318912,0.440963,0.228312,0.921815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.634409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.431170,0.350087,0.318912,0.440963,0.228312,0.921815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.709677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.431170,0.350087,0.318912,0.440963,0.228312,0.921815,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11164,0.0,0.903226,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.431170,0.350087,0.318912,0.440963,0.228312,0.921815,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
11165,0.0,0.473118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.431170,0.350087,0.318912,0.440963,0.228312,0.921815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11166,0.0,0.376344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.449438,0.245455,0.200000,0.353535,0.090909,0.916667,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11167,0.0,0.322581,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.438202,0.472727,0.343750,0.464646,0.181818,0.921815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
np.random.seed(50)

trainSet = np.random.choice(len(df_combined), round(len(df_combined)*0.70), replace=False)
valSet = np.random.choice(len(df_combined), round(len(df_combined)*0.15), replace=False)
testSet = np.random.choice(len(df_combined), round(len(df_combined)*0.15), replace=False)

trainData = df_.iloc[trainSet]
valData = df_.iloc[valSet]
testData = df_.iloc[testSet]

print("Training:", len(trainData))
print("Validation:", len(valData))
print("Testing:", len(testData))

Training: 7818
Validation: 1675
Testing: 1675
